# NumPy Array Shape Manipulation 

This notebook covers reshaping, flattening, transposing, changing dimensions and memory order


## Reshaping Arrays — `reshape()`

- Changes the **view** when possible; otherwise returns a **copy**.
- You may use `-1` to infer one dimension size.
- Use `order='C'` (row-major) or `order='F'` (column-major) for reading/writing order semantics during reshape.


In [ ]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)

a = np.arange(12)
b = a.reshape(3, 4)           # likely a view
c = a.reshape(2, -1)          # -1 infers size 6 -> (2, 6)

print('a shape:', a.shape, '| b shape:', b.shape, '| c shape:', c.shape)
a[0] = -1                     # mutate original to check view semantics
print('\na after edit:', a)
print('b reflects change?\n', b)

**Reshape with different orders**  
`order='F'` interprets data in column-major order for the transformation (without changing underlying storage unless a copy is needed).

In [ ]:
x = np.arange(1, 13)  # 1..12
xC = x.reshape(3, 4, order='C')
xF = x.reshape(3, 4, order='F')

print('order=C reshape (3x4):\n', xC)
print('\norder=F reshape (3x4):\n', xF)

## Flattening — `ravel()` vs `flatten()`

- `ravel()` returns a **flattened view** if possible (no copy).
- `flatten()` always returns a **copy**.
- Both support `order='C'` or `order='F'`.


In [ ]:
M = np.arange(1, 13).reshape(3, 4)
r = M.ravel()                 # view when possible
f = M.flatten()               # copy

print('M:\n', M)
print('\nravel view?:', r.flags['OWNDATA'] == False)
print('flatten copy?:', f.flags['OWNDATA'] == True)

M[0,0] = 999  # mutate to see which one reflects the change
print('\nAfter editing M[0,0]=999')
print('r (ravel):', r)   # likely reflects change
print('f (flatten):', f) # remains original

# Order demo
print('\nravel(order=F):', M.ravel(order='F'))

## Transposing — `.T`, `transpose()`, `swapaxes()`

- `.T` is shorthand for reversing axes for 2D arrays.
- `np.transpose(a, axes=...)` allows explicit axis permutation.
- `np.swapaxes(a, axis1, axis2)` swaps two axes.


In [ ]:
A = np.arange(1, 13).reshape(3, 4)
print('A shape:', A.shape)
print('A.T shape:', A.T.shape)
print('A.T:\n', A.T)

B = np.arange(24).reshape(2, 3, 4)     # (d0, d1, d2)
BT = np.transpose(B, (1, 0, 2))        # reorder axes to (d1, d0, d2)
print('\nB shape:', B.shape, '-> transpose(1,0,2):', BT.shape)

BS = np.swapaxes(B, 0, 2)              # swap first and last axes
print('swapaxes(0,2) shape:', BS.shape)

## Adding / Removing Dimensions — `newaxis`/`None`, `expand_dims()`, `squeeze()`

- Add a new length-1 axis with `a[np.newaxis, ...]` or `None`.
- `np.expand_dims(a, axis=k)` inserts a new axis at position `k`.
- `np.squeeze(a, axis=None)` removes axes of length 1 (optionally specific axis).


In [ ]:
v = np.arange(5)                 # (5,)
row = v[np.newaxis, :]            # (1,5)
col = v[:, np.newaxis]            # (5,1)

print('v shape:', v.shape)
print('row shape:', row.shape, '| col shape:', col.shape)

w = np.expand_dims(v, axis=0)     # (1,5)
u = np.expand_dims(v, axis=1)     # (5,1)
print('expand_dims axis=0:', w.shape, '| axis=1:', u.shape)

t = np.array([[[1,2,3]]])         # shape (1,1,3)
print('\nBefore squeeze:', t.shape)
ts = np.squeeze(t)                # shape (3,)
print('After squeeze:', ts.shape)

# selective squeeze
t2 = np.zeros((1, 3, 1, 4))
print('\nSelective squeeze on axis=2:')
print('before:', t2.shape, 'after:', np.squeeze(t2, axis=2).shape)

## Changing Array Order — `order='C'` vs `order='F'`

- **C-order (row-major):** last index changes fastest in memory.
- **F-order (column-major):** first index changes fastest.
- Affect how data is **interpreted** or **copied** by operations like `reshape`, `ravel`, `copy`.


In [ ]:
Z = np.arange(1, 13).reshape(3, 4, order='C')
ZF = np.array(Z, order='F')             # ensure Fortran-ordered copy

print('Z C_CONTIGUOUS:', Z.flags['C_CONTIGUOUS'], '| F_CONTIGUOUS:', Z.flags['F_CONTIGUOUS'])
print('ZF C_CONTIGUOUS:', ZF.flags['C_CONTIGUOUS'], '| F_CONTIGUOUS:', ZF.flags['F_CONTIGUOUS'])

# ravel obeys order parameter when flattening
print('\nravel C-order:', ZF.ravel(order='C'))   # reads rows first
print('ravel F-order:', ZF.ravel(order='F'))     # reads columns first

# reshape with order affects mapping from 1D to N-D
u = np.arange(12)
uC = np.reshape(u, (3,4), order='C')
uF = np.reshape(u, (3,4), order='F')
print('\nreshape order=C:\n', uC)
print('reshape order=F:\n', uF)

## Views vs Copies — quick checks

- Many shape-manipulation ops return **views** when possible. Use `.base is not None` or `OWNDATA` flag to reason about ownership.


In [ ]:
base = np.arange(9).reshape(3,3)
view1 = base.T                # transpose is a view
view2 = base.ravel()          # often a view
copy1 = base.flatten()        # copy by definition

print('base owns data?      ', base.flags['OWNDATA'])
print('T owns data?         ', view1.flags['OWNDATA'], '| shares base?', view1.base is base)
print('ravel owns data?     ', view2.flags['OWNDATA'])
print('flatten owns data?   ', copy1.flags['OWNDATA'])